# Basic Arnoldi

From ChatGPT precursor

In [41]:
using LinearAlgebra
function ArnoldiStd(A, v, m)            # A is nxn and v is an n vector 
    n = size(A,1)
    Q = Matrix{Float64}(undef, n, m+1)  # Orthogonal vectors
    H = zeros(Float64, m+1, m)          # Upper Hessenberg matrix
    # Store v in first column of Q
    Q[:, 1] = v
    for k = 1:m # Arnoldi iteration
        w = A*Q[:, k]
        for j = 1:k # Orthogonalization using modified Gram-Schmidt
            H[j, k] = dot(Q[:, j], w)
            w -= H[j, k] * Q[:, j]
        end
        H[k+1, k] = norm(w)
        Q[:, k+1] = w / H[k+1, k]
    end
    return Q, H
end

ArnoldiStd (generic function with 1 method)

## AD Directional Derivative Code
Forward Diff was abandoned in favor of writing a self contained efficiient code. Below is the first derivative code. The first column of B contains v. The remainng columns contains b directions perpendicular to v.

In [74]:
using LinearAlgebra
function ArnoldiDD(A, v, dv, m)            # A is nxn and v and dv are n vectors 
    n = size(A,1)
    Q = zeros(Float64, n, m+1)  # Orthogonal vectors
    dQ= zeros(Float64, n, m+1)  # Directional Derivative of Q
    H = zeros(Float64, m+1, m)  # Upper Hessenberg matrix
    dH= zeros(Float64, m+1, m)  # Directional Derivative of H
    # Store v and dv in first column of Q and dQ
     Q[:, 1] = v
    dQ[:, 1] = dv
    for k = 1:m # Arnoldi iteration
        w = A*Q[:, k]; dw=A*dQ[:,k]
        for j = 1:k # Orthogonalization using modified Gram-Schmidt
            dH[j, k] = dot(dQ[:, j], w)+dot(Q[:, j], dw)
            H[j, k] = dot(Q[:, j], w)
            dw -=dH[j,k]*Q[:,j]+H[j,k]*dQ[:,j]
            w -= H[j, k]*Q[:, j]
        end
        dH[k+1, k] = dot(w,dw)/norm(w)
        H[k+1, k] = norm(w)
        dQ[:,k+1] = dw/H[k+1,k]- w*dH[k+1,k]/H[k+1,k]^2
        Q[:, k+1] = w / H[k+1, k]
    end
    return Q, dQ, H, dH
end

ArnoldiDD (generic function with 1 method)

In [102]:
using MMGet, SparseArrays
A = mmget("https://math.nist.gov/pub/MatrixMarket2/Harwell-Boeing/bcsstruc1/bcsstk01.mtx.gz", keep_files = false)
(b, m) = (1,12)
n = size(A,1)
B = (qr(rand(n,b+1)).Q)*Matrix(1.0I, b+1, b+1) # The Q field converts to thick by default
(v, dv) = (B[:,1], B[:,2])
(Q,H) = ArnoldiStd(A, v, m)
(Qn,dQn,Hn,dHn) = ArnoldiDD(A, v, dv, m)
# A*Q[:,1:m] = Q*H so A*dQ[:,1:m] = dQ*H+Q*dH
# Q'*Q = I so dQ'*Q + Q'*dQ = 0 i.e W=Q'*dQ is skew
W = Qn'*dQn
(norm(W+W'),norm(dQn*H+Qn*dHn-A*dQn[:,1:m])/norm(A*dQn))
# Residuals check out appropriately. 

(1.6891950538468234e-13, 1.7961591862316718e-16)

## More Directions
The next thing to do is to add additional directions.  

In [163]:
using LinearAlgebra
function ArnoldiGrad(A, v, dV, m)       # A is nxn, v is an n vector, and dV is an nxb array 
    n = size(A,1); b=size(dV,2)
    Q = zeros(Float64, n, m+1)  # Orthogonal vectors
    dQ= zeros(Float64, n, m+1, b)  # Directional Derivatives of Q
    H = zeros(Float64, m+1, m)  # Upper Hessenberg matrix
    dH= zeros(Float64, m+1, m, b)  # Directional Derivative of H
    # Store v in first column of Q and dV in dQ
    # writing out 1:b for clarity
     Q[:,1] = v
    dQ[:,1,1:b] = dV
    for k = 1:m # Arnoldi iteration
        w = A*Q[:, k]; dw=A*dQ[:,k,1:b]
        for j = 1:k # Orthogonalization using modified Gram-Schmidt
            H[j, k] = w'*Q[:, j]
            for bb in 1:b
                dH[j, k, bb] = w'*dQ[:, j, bb] + Q[:, j]'*dw[:,bb]
                dw[:,bb] -=dH[j,k,bb]*Q[:,j]+H[j,k]*dQ[:,j,bb]
            end
            w -= H[j, k]*Q[:, j]
        end
        H[k+1, k] = norm(w)
        # Updating dH
        for bb in 1:b
            dH[k+1, k,bb] = w'*dw[:,bb]/norm(w)
            dQ[:,k+1,bb] = dw[:,bb]/H[k+1,k]- w*dH[k+1,k,bb]/H[k+1,k]^2
        end
        Q[:, k+1] = w / H[k+1, k]
    end
    return Q, dQ, H, dH
end

ArnoldiGrad (generic function with 1 method)

In [164]:
using MMGet, SparseArrays
A = mmget("https://math.nist.gov/pub/MatrixMarket2/Harwell-Boeing/bcsstruc1/bcsstk01.mtx.gz", keep_files = false)
(b, m) = (4,5)
n = size(A,1)
B = (qr(rand(n,b+1)).Q)*Matrix(1.0I, b+1, b+1) # The Q field converts to thick by default
(v, dV) = (B[:,1], B[:,2:b+1])
(Q,H) = ArnoldiStd(A, v, m)
(Qn,dQn,Hn,dHn) = ArnoldiGrad(A, v, dV, m)
# A*Q[:,1:m] = Q*H so A*dQ[:,1:m,bb] = dQ[:,:,bb]*H+Q*dH[:,:,bb]
# Q'*Q = I so dQ'*Q + Q'*dQ = 0 i.e W=Q'*dQ[:,:,bb] is skew
for bb in 1:b
    dQ = dQn[:,:,bb]
    dH = dHn[:,:,bb]
    W= Qn'*dQ
    SkewNorm=norm(W+W')
    ResNorm =norm(dQ*H+Qn*dH-A*dQ[:,1:m])/norm(A*Qn) 
    display((bb,SkewNorm,ResNorm))
end
# Residuals check out appropriately. 

(1, 5.451953497470235e-15, 1.3351636685026925e-16)

(2, 1.4755992111031775e-15, 8.613382833727093e-17)

(3, 6.562571889100335e-15, 8.290973954752038e-17)

(4, 1.4093795945673038e-15, 9.334826963587998e-17)

## Break
Checking that I understand the contraction order. 

In [141]:
using LinearAlgebra
v1=[1 2 4]
v2=[3 4 5]
(dot(v1,v2),v1*v2',v1'*v2,v2'*v1)

(31, [31;;], [3 4 5; 6 8 10; 12 16 20], [3 6 12; 4 8 16; 5 10 20])

In [38]:
function ArnoldiAD1(A, B, m) #B is assumed to be an n
    
    n = length(v)
    Q = Matrix{Float64}(undef, n, m+1)  # Matrix to store orthogonalized vectors
    H = zeros(Float64, m+1, m)          # Upper Hessenberg matrix

    # Skipping normalizing initial vector and store it in Q
    Q[:, 1] = v

    for k = 1:m
        # Arnoldi iteration
        w = A*Q[:, k]
        for j = 1:k
            # Orthogonalization using modified Gram-Schmidt
            H[j, k] = dot(Q[:, j], w)
            w -= H[j, k] * Q[:, j]
        end
        H[k+1, k] = norm(w)
        Q[:, k+1] = w / H[k+1, k]
    end
    return Q, H

Arnoldi3 (generic function with 1 method)

## Messed Up 

In [60]:
using LinearAlgebra
function ArnoldiADGrad(A, B, m)         # A is nxn and B is a nx(b+1)
    (n,b) = (size(A,1),size(B,2)-1)
    QdQ = zeros(Float64, n, b+1, m+1)     # Orthogonal vectors and 1st Derivatives 
    HdH = zeros(Float64, b+1, m+1, m)     # Upper Hessenberg matrix and 1st Derivatives
    # Storing initial vectors
    QdQ[:,:,1] = B
    for k = 1:m # Arnoldi iteration
        wdw = A*QdQ[:,:,k]
        w = wdw[:,k]
        for j = 1:k
            # Orthogonalization using modified Gram-Schmidt
            HdH[1,j, k] = dot(QdQ[:,1, j], w)
            w-= HdH[1,j, k] * QdQ[:,1, j]
        end
        HdH[1,k+1, k] = norm(w)
        QdQ[:,1, k+1] = w / HdH[1,k+1, k]
    end
    return QdQ[:,1,:], HdH[1,:,:] #AAS Friday 3rd Should be original Q and H
end

ArnoldiADGrad (generic function with 1 method)

Testing standard input output form using a package from https://github.com/CHLOzzz/MMGet/blob/main/README.md to grab a MatrixMarket matrix.  keep_files = true preserves files. 

In [72]:
using MMGet, SparseArrays
A = mmget("https://math.nist.gov/pub/MatrixMarket2/Harwell-Boeing/bcsstruc1/bcsstk01.mtx.gz", keep_files = false)
(b, m) = (3,4)
n = size(A,1)
B = (qr(rand(n,b+1)).Q)*Matrix(1.0I, b+1, b+1) # The Q field converts to thick by default
(Qn,Hn)=ArnoldiADGrad(A, B, m)
(Q,H) = ArnoldiStd(A, B[:,1], m)
Qn-Q
Hn-H

5×4 Matrix{Float64}:
 0.0  -1.04475e9   -7.7486e-7    2.19047e-6
 0.0  -1.59788e9   -7.25979e8    3.17395e-6
 0.0  -7.25979e8  NaN          NaN
 0.0   0.0        NaN          NaN
 0.0   0.0          0.0        NaN

In [6]:
# @time (Q, H) = Arnoldi1(A,v,m)
# (norm(A*Q[:,1:m] - Q*H)/norm(A),norm(Q'*Q-I),H[m+1,m])

LoadError: DimensionMismatch: first dimension of B, 2, must equal one of the dimensions of Q, (48, 48)

Testing reduced output form.  Only bottom entry of H. 

In [131]:
using MMGet, LinearAlgebra, SparseArrays
A = mmget("https://math.nist.gov/pub/MatrixMarket2/Harwell-Boeing/bcsstruc1/bcsstk01.mtx.gz", keep_files = false)
n=size(A,1)
#A=rand(n,n)
function f(y)
    
    n = 48
    Q = Matrix{Float64}(undef, n, m+1)  # Matrix to store orthogonalized vectors
    H = zeros(Float64, m+1, m)          # Upper Hessenberg matrix

    # Normalizing initial vector and store it in Q
    v = b + G*y
    Q[:, 1] = v/sqrt(v'*v)
    for k = 1:m
        # Arnoldi iteration
        w = A*Q[:, k]
        for j = 1:k
            # Orthogonalization using modified Gram-Schmidt
            H[j, k] = Q[:, j]'*w
            w -= H[j, k] * Q[:, j]
        end
        H[k+1, k] = sqrt(w'*w)
        Q[:, k+1] = w / H[k+1, k]
    end
    return H[m+1,m]
end

f (generic function with 1 method)

In [133]:
y=zeros(s)
f(y)
ForwardDiff.gradient(f,y)

LoadError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{typeof(f), Float64}, Float64, 3})

[0mClosest candidates are:
[0m  (::Type{T})(::Real, [91m::RoundingMode[39m) where T<:AbstractFloat
[0m[90m   @[39m [90mBase[39m [90m[4mrounding.jl:207[24m[39m
[0m  (::Type{T})(::T) where T<:Number
[0m[90m   @[39m [90mCore[39m [90m[4mboot.jl:792[24m[39m
[0m  Float64([91m::UInt8[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mfloat.jl:165[24m[39m
[0m  ...
